In [1]:
import pandas as pd
import numpy as np
import os
import re
import random
import pandas as pd

# Loading Product Review Dataset

In [2]:
df = pd.read_csv("aws_review_sofware_dataset.csv")

C:\Users\91798\AppData\Local\Temp\ipykernel_22976\2031363380.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("aws_review_sofware_dataset.csv")


In [3]:
df.columns

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')

In [4]:
df=df.sample(n=10000,random_state=42)
df=df[df.index.isin(df.index)]
df=df.reset_index(drop=True)

In [5]:
df.isnull().sum()

Unnamed: 0           0
overall              0
verified             0
reviewTime           0
reviewerID           0
asin                 0
style             4930
reviewerName         1
reviewText           0
summary              1
unixReviewTime       0
vote              7143
image             9965
dtype: int64

In [6]:
df["words"]="default value"
df["sentences"]="default value"
for i in range(df.shape[0]):
    df.at[i,"words"]= list("")
    df.at[i,"sentences"] = list("")

In [7]:
df.columns

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image', 'words', 'sentences'],
      dtype='object')

# Sentence Tokenization

In [8]:
from nltk.tokenize import sent_tokenize

In [9]:
for i in range(10000):
    l1= sent_tokenize(str(df.loc[i,"reviewText"]))
    df.at[i,"sentences"]=l1

# Word Tokenization

In [10]:
from pywsd.utils import lemmatize_sentence

Warming up PyWSD (takes ~10 secs)... took 4.404162645339966 secs.


In [11]:
for k in range(10000):
    df.at[k,"words"]=list("")
    for j in range(len(df.loc[k,"sentences"])):
        df.at[k,"words"].extend(lemmatize_sentence(df.loc[k,"sentences"][j]))

In [12]:
df["words_sentences"] = [' ' for i in range(10000)]

In [13]:
def get_str(list1):
    s1 = ''
    for i1 in list1:
        s1 = s1+' '+i1
    return s1

In [14]:
import functools
for k in range(df.shape[0]):
    df.loc[k,"words_sentences"]=get_str(df.loc[k,"words"])

# Count Vectorizer

In [15]:
from sklearn.feature_extraction.text import  CountVectorizer

In [16]:
df1=df

no_features = 1000
tf_vectorizer = CountVectorizer( max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(df1.words_sentences)

In [17]:
df_x=pd.DataFrame(tf.A, columns=tf_vectorizer.get_feature_names_out())

In [18]:
df_y=df["verified"]

# Label Encoding

In [19]:
from sklearn.preprocessing import LabelEncoder

In [20]:
le=LabelEncoder()
df_y_1=pd.DataFrame(df_y)
df_y_enc=df_y_1.apply(le.fit_transform)

In [21]:
df_y_enc.columns

Index(['verified'], dtype='object')

In [22]:
df_y_enc.head(5)

,verified
0,1
1,1
2,1
3,1
4,1


# Decision Tree Classifier

In [23]:
from sklearn.tree import DecisionTreeClassifier

In [24]:
DecTree=DecisionTreeClassifier()
dt_M=DecTree.fit(df_x,df_y_enc)
dt_M.score(df_x,df_y_enc)

0.9952